In [42]:
import simpy
import random

In [43]:
# Create a Hospital class. It has limited beds ('NUM_BEDS') to treat patients in parallel.
# Patients wait for an available bed. When they get one they occupy it for a random time between 1 and 10.

class Hospital(object):
    def __init__(self, env, NUM_BEDS):
        self.env = env
        self.bed = simpy.Resource(env, NUM_BEDS)

    def treatment(self, patient):
        yield self.env.timeout(random.randint(1, 10))
        print("%d%% probability of %s returning to hospital" %
              (random.randint(1, 99), patient))

In [44]:
# Defines a patient which requires a bed upon arriving at hospital. 

def patient(env, name, cw):
    print('%s arrives at the hospital at %.2f.' % (name, env.now))
    with cw.bed.request() as request:
        yield request

        print('%s enters the hospital at %.2f.' % (name, env.now))
        yield env.process(cw.treatment(name))

        print('%s leaves the hospital at %.2f.' % (name, env.now))

In [45]:
# create the enviroment with 4 initial patients and keep generating more patients every random
# interval between 1 and 5 while the simulation is running.

def setup(env, NUM_BEDS):
    hospital = Hospital(env, NUM_BEDS)

    for i in range(4):
        env.process(patient(env, 'Patient %d' % i, hospital))

    while True:
        yield env.timeout(random.randint(1, 5))
        i += 1
        env.process(patient(env, 'Patient %d' % i, hospital))

In [46]:
# input parameters

NUM_BEDS = 4  
SIM_TIME = 20 # days, for example


# initialise enviroment

env = simpy.Environment()
env.process(setup(env, NUM_BEDS))

<Process(setup) object at 0x10b9252e8>

In [47]:
# execute simulation

env.run(until=SIM_TIME)

Patient 0 arrives at the hospital at 0.00.
Patient 1 arrives at the hospital at 0.00.
Patient 2 arrives at the hospital at 0.00.
Patient 3 arrives at the hospital at 0.00.
Patient 0 enters the hospital at 0.00.
Patient 1 enters the hospital at 0.00.
Patient 2 enters the hospital at 0.00.
Patient 3 enters the hospital at 0.00.
11% probability of Patient 1 returning to hospital
Patient 1 leaves the hospital at 1.00.
Patient 4 arrives at the hospital at 2.00.
13% probability of Patient 0 returning to hospital
Patient 4 enters the hospital at 2.00.
Patient 0 leaves the hospital at 2.00.
36% probability of Patient 2 returning to hospital
Patient 5 arrives at the hospital at 4.00.
Patient 2 leaves the hospital at 4.00.
Patient 5 enters the hospital at 4.00.
21% probability of Patient 3 returning to hospital
Patient 3 leaves the hospital at 5.00.
Patient 6 arrives at the hospital at 8.00.
Patient 6 enters the hospital at 8.00.
27% probability of Patient 4 returning to hospital
Patient 4 leave